In [4]:
from datasets import load_dataset

In [5]:
lang_code = "fa_ir"
dataset = load_dataset("google/fleurs", lang_code, split="test", trust_remote_code=True)

Using the latest cached version of the module from /home/jovyan/.cache/huggingface/modules/datasets_modules/datasets/google--fleurs/80cb68d1b4d319aefbd8ea302274d3950d95f6242f0742c1452d1545c80a2d5f (last modified on Sat Jun 28 09:10:41 2025) since it couldn't be found locally at google/fleurs, or remotely on the Hugging Face Hub.


{'id': 1665, 'num_samples': 271680, 'path': '/home/dev/.cache/huggingface/datasets/downloads/extracted/c8482b7f4e027e4138a0d9e0cfc67334cfa9e1617b08e6e3ba3cbbcc5165c116/10039392309700583023.wav', 'audio': {'path': 'test/10039392309700583023.wav', 'array': array([0.        , 0.        , 0.        , ..., 0.0005191 , 0.00224453,
       0.00388223]), 'sampling_rate': 16000}, 'transcription': 'این سند بر اساس این درز اطلاعاتی است و به اختلاف مرزی مربوط می\u200cشود که در آن فلسطین خواستار مرزبندی طبق شرایط قبل از جنگ شش\u200cروزه ۱۹۶۷ است', 'raw_transcription': 'این سند بر اساس این درز اطلاعاتی است و به اختلاف مرزی مربوط می\u200cشود، که در آن فلسطین خواستار مرزبندی طبق شرایط قبل از جنگ شش\u200cروزه 1967 است.', 'gender': 0, 'lang_id': 22, 'language': 'Persian', 'lang_group_id': 2}


In [16]:
import IPython.display as ipd

for i in range(3):
    row = dataset[i]
    for key in row:
        print(f"{key}: {row[key]}")
    ipd.display(ipd.Audio(data=row["audio"]["array"], rate=row["audio"]["sampling_rate"]))
    print("-" * 80)

id: 1665
num_samples: 271680
path: /home/dev/.cache/huggingface/datasets/downloads/extracted/c8482b7f4e027e4138a0d9e0cfc67334cfa9e1617b08e6e3ba3cbbcc5165c116/10039392309700583023.wav
audio: {'path': 'test/10039392309700583023.wav', 'array': array([0.        , 0.        , 0.        , ..., 0.0005191 , 0.00224453,
       0.00388223]), 'sampling_rate': 16000}
transcription: این سند بر اساس این درز اطلاعاتی است و به اختلاف مرزی مربوط می‌شود که در آن فلسطین خواستار مرزبندی طبق شرایط قبل از جنگ شش‌روزه ۱۹۶۷ است
raw_transcription: این سند بر اساس این درز اطلاعاتی است و به اختلاف مرزی مربوط می‌شود، که در آن فلسطین خواستار مرزبندی طبق شرایط قبل از جنگ شش‌روزه 1967 است.
gender: 0
lang_id: 22
language: Persian
lang_group_id: 2


--------------------------------------------------------------------------------
id: 1966
num_samples: 179520
path: /home/dev/.cache/huggingface/datasets/downloads/extracted/c8482b7f4e027e4138a0d9e0cfc67334cfa9e1617b08e6e3ba3cbbcc5165c116/10051486205118679596.wav
audio: {'path': 'test/10051486205118679596.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00016022,
       -0.00015008, -0.00018591]), 'sampling_rate': 16000}
transcription: این کمیسیون پاسخ مارتلی بود به اعتراضات گسترده ضدحکومتی که در ماه اکتبر آغاز شد
raw_transcription: این کمیسیون پاسخ مارتلی بود به اعتراضات گسترده ضدحکومتی که در ماه اکتبر آغاز شد.
gender: 0
lang_id: 22
language: Persian
lang_group_id: 2


--------------------------------------------------------------------------------
id: 1903
num_samples: 220800
path: /home/dev/.cache/huggingface/datasets/downloads/extracted/c8482b7f4e027e4138a0d9e0cfc67334cfa9e1617b08e6e3ba3cbbcc5165c116/1010306930168345338.wav
audio: {'path': 'test/1010306930168345338.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -6.93798065e-05, -6.25848770e-05, -7.20024109e-05]), 'sampling_rate': 16000}
transcription: زمانی که از همه منابع موجود به شکلی مؤثر در واحدهای عملیاتی سازمانی استفاده شود خلاقیت و نبوغ می‌تواند اتفاق افتد
raw_transcription: زمانی که از همه منابع موجود به شکلی مؤثر در واحدهای عملیاتی سازمانی استفاده شود، خلاقیت و نبوغ می‌تواند اتفاق افتد.
gender: 0
lang_id: 22
language: Persian
lang_group_id: 2


--------------------------------------------------------------------------------


In [2]:
import torch
from datasets import load_dataset
from transformers import pipeline
from jiwer import wer
from tqdm import tqdm

In [ ]:
# Optional: to save or play audio
import soundfile as sf
from IPython.display import Audio, display

# --- CONFIG ---
device = "cuda" if torch.cuda.is_available() else "cpu"
lang_code = "fa_ir"
num_samples = 20  # use more for full evaluation
save_audio = False  # set to True to save files
play_audio = False  # set to True in notebooks to play files

models = [
    "openai/whisper-small",
    "m3hrdadfi/wav2vec2-large-xlsr-persian"
]

# --- Load Persian FLEURS dataset ---
print("Loading dataset...")
dataset = load_dataset("google/fleurs", lang_code, split=f"test[:{num_samples}]")

# --- Evaluation loop ---
for model_name in models:
    print(f"\n🔍 Evaluating model: {model_name}")
    asr = pipeline("automatic-speech-recognition", model=model_name, device=0 if device == "cuda" else -1)

    references, hypotheses = [], []

    for idx, sample in tqdm(enumerate(dataset), total=len(dataset)):
        audio_array = sample['audio']['array']
        sampling_rate = sample['audio']['sampling_rate']
        reference_text = sample['transcription'].strip().lower()

        try:
            # --- Run ASR ---
            result = asr(audio_array, chunk_length_s=30, return_timestamps=False)
            predicted_text = result["text"].strip().lower()

            # --- Collect results ---
            references.append(reference_text)
            hypotheses.append(predicted_text)

            # --- Optional: Save or play audio ---
            if save_audio:
                out_path = f"audio_{idx}.wav"
                sf.write(out_path, audio_array, sampling_rate)
            if play_audio:
                display(Audio(data=audio_array, rate=sampling_rate))

        except Exception as e:
            print(f"Error on sample {idx}: {e}")

    # --- Calculate and display WER ---
    error = wer(references, hypotheses)
    print(f"✅ WER for {model_name} on FLEURS (Persian): {error:.3f}")

IterableDataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_shards: 1
})